In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pickle
import json
from time_series_split import *

In [ ]:
def calculate_aqi(pm25):
    ranges = [
        (0.0, 12.0, 0, 50),
        (12.1, 35.4, 51, 100),
        (35.5, 55.4, 101, 150),
        (55.5, 150.4, 151, 200),
        (150.5, 250.4, 201, 300),
        (250.5, 500.0, 301, 500),
    ]
    for c_low, c_high, aqi_low, aqi_high in ranges:
        if c_low <= pm25 <= c_high:
            return round((aqi_high - aqi_low) / (c_high - c_low) * (pm25 - c_low) + aqi_low)
    return 500  # default nếu vượt ngưỡng

def aqi_class(aqi):
    if aqi <= 50: return 0
    elif aqi <= 100: return 1
    elif aqi <= 150: return 2
    elif aqi <= 200: return 3
    elif aqi <= 300: return 4
    else: return 5

In [ ]:
combined_data = pd.read_csv('/home/thu/INT3041E_AI_PM2.5-Concentration-Estimation/data/add_AQI.csv')
combined_data['AQI'] = combined_data['pm25'].apply(calculate_aqi)
combined_data['AQI_Class'] = combined_data['AQI'].apply(aqi_class)

In [ ]:
combined_data.head()

In [ ]:
# folds = split_original_data()
folds = split_consolidated_data()
print(f"Number of folds: {len(folds)}")

In [ ]:
fold = folds[0]
train_data = fold['train']
val_data = fold['validation']
test_data = fold['test']

In [ ]:
# Chuẩn bị features và target từ các DataFrame
# Features: Loại trừ pm25, AQI, AQI_Class (tương tự cột 2 đến -2 trong combined_data)
feature_columns = train_data.columns[3:-2]  # Từ cột 3 (lat) đến cột -2 (AAI)

X_train = train_data[feature_columns]
y_train = train_data['AQI_Class']

X_val = val_data[feature_columns]
y_val = val_data['AQI_Class']

X_test = test_data[feature_columns]
y_test = test_data['AQI_Class']

In [ ]:
feature_columns

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42) 
X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

## Tuning parameters

In [ ]:
strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=43)

**RandomForest**

n_estimators: Tăng phạm vi từ 100-1200 lên 100-1500 và thêm nhiều giá trị hơn.
max_features: Thêm giá trị None để thử nghiệm sử dụng toàn bộ các feature.
max_depth: Tăng phạm vi từ 5-30 lên 5-50 và thêm giá trị None để không giới hạn độ sâu.
min_samples_split: Thêm giá trị 15 để thử nghiệm với các tập dữ liệu lớn hơn.
min_samples_leaf: Thêm giá trị 8 để kiểm tra các cây có lá lớn hơn.
bootstrap: Thêm tham số này để thử nghiệm với cả hai chế độ bootstrap (True và False).
n_iter: Tăng số lần thử nghiệm từ 50 lên 100 để cải thiện khả năng tìm kiếm.

In [ ]:
rf_params = {
    'n_estimators': [int(x) for x in np.linspace(100, 1500, 15)],  # Tăng phạm vi n_estimators
    'max_features': ['sqrt', 'log2', None],  # Thêm None để thử toàn bộ các feature
    'max_depth': [int(x) for x in np.linspace(5, 50, 10)] + [None],  # Thêm None để không giới hạn độ sâu
    'min_samples_split': [2, 5, 10, 16],  
    'min_samples_leaf': [1, 2, 4, 8],  
    'bootstrap': [True, False]  # Thêm lựa chọn bootstrap
}

rf_model = RandomForestClassifier(random_state=43, class_weight='balanced')
rf_search = RandomizedSearchCV(
    rf_model, 
    rf_params, 
    cv=strat_kfold, 
    scoring='accuracy', 
    n_iter=100,  # Tăng số lần thử nghiệm
    n_jobs=-1, 
    random_state=43
)
rf_search.fit(X_train, y_train)
rf_val_acc = rf_search.score(X_val, y_val)

In [ ]:
all_labels = [0, 1, 2, 3, 4, 5]
target_names = ['Good', 'Moderate', 'Unhealthy for Sensitive', 'Unhealthy', 'Very Unhealthy', 'Hazardous']

cm = confusion_matrix(y_test, pred, labels=all_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names)
disp.plot(xticks_rotation=45)
plt.title(f'{best_model_name} - Confusion Matrix')
plt.show()

In [ ]:
print(f"Test Accuracy: {metrics.accuracy_score(y_test, pred):.4f}")
print("Classification Report (Test Set):")
print(metrics.classification_report(y_test, pred, labels=all_labels, target_names=target_names, zero_division=1))

In [ ]:
# Feature Importance (nếu hỗ trợ)
try:
    importances = best_clf.feature_importances_
    indices = np.argsort(importances)[::-1]
    plt.figure(figsize=(10, 6))
    sns.barplot(x=importances[indices], y=X_train.columns[indices])
    plt.title(f'{best_model_name} - Feature Importances')
    plt.xlabel('Importance Score')
    plt.ylabel('Features')
    plt.tight_layout()
    plt.show()
except:
    print("This model does not support feature_importances_.")

In [ ]:
# Lưu model & metadata

pickle.dump(best_clf, open(f'{best_model_name.lower()}-classifier-tuning.pkl', 'wb'))
metadata = {
    "model_name": best_model_name,
    "best_params": best_search.best_params_,
    "val_accuracy": best_val_acc,
    "test_accuracy": metrics.accuracy_score(y_test, pred),
    "features": list(feature_columns),
    "classes": sorted(list(set(y_train)))
}
with open(f"{best_model_name.lower()}_metadata.json", "w") as f:
    json.dump(metadata, f, indent=4)